In [85]:
def getLoss(CostofRepair,CorporateIncomeLoss,OrganizationProductiveLoss,CostofEquipmentReplacement,SLALoss,IndirectLoss,TimetoRepair_without,TimetoDetect_without):
    L1 = CostofRepair + CorporateIncomeLoss +OrganizationProductiveLoss
    L2 = CorporateIncomeLoss + OrganizationProductiveLoss
    L3 = CostofEquipmentReplacement + SLALoss + IndirectLoss
    L = L1 * TimetoRepair_without + L2 * TimetoDetect_without + L3
    return L1,L2,L3,L

def getRisk(TimesofAttack,TimePeriod,TimesofSuccessfulAttack,L):
    T = TimesofAttack/12
    v = TimesofSuccessfulAttack/TimesofAttack
    R0 = T*v*L
    return R0

def getCost(InitialCost,AnnualUpgrade,AnnualMaintenance,TimePeriod):
    TotalCost = InitialCost+(AnnualUpgrade+AnnualMaintenance)*TimePeriod/12
    return TotalCost

def getMeasureTypeID(R0,R_min,R_max,L_max):
    if R0 < R_min:
        MeasureTypeID = 0
    elif R0 > R_max:
        MeasureTypeID = 3
    elif L > L_max:
        MeasureTypeID = 2
    else:
        MeasureTypeID = 1
    return MeasureTypeID

def calculateROSI(MeasureTypeID,TimesofAttack,TimePeriod,TimesofSuccessfulAttack,ReducedVulnerability,InsuranceAmount,TotalCost,R0,L1,L2,L3,TimetoRepair_with,TimetoDetect_with):
    T = TimesofAttack/12
    v = TimesofSuccessfulAttack/TimesofAttack
    vc = (1-ReducedVulnerability)*v
    if MeasureTypeID == 0:
        ROSI = -1.0
    elif MeasureTypeID == 3:
        ROSI = 0.0
    elif MeasureTypeID == 2:
        ROSI = (T*v*InsuranceAmount-TotalCost)/TotalCost
    else:
        RC = T*vc*(L1*TimetoRepair_with+L2*TimetoDetect_with+L3)
        ROSI = (R0-RC-TotalCost)/TotalCost
    return ROSI

def calculateNPV(MeasureTypeID,TimesofAttack,TimePeriod,TimesofSuccessfulAttack,ReducedVulnerability,InsuranceAmount,TotalCost,R0,L1,L2,L3,TimetoRepair_with,TimetoDetect_with,DiscountRate):
    T = TimesofAttack/(TimePeriod/12)
    v = TimesofSuccessfulAttack/TimesofAttack
    vc = (1-ReducedVulnerability)*v
    year = int(TimePeriod/12)
    NPV=0.0
    if TimePeriod >= 12:
        if MeasureTypeID == 0 or MeasureTypeID == 3:
            NPV = 0.0
        elif MeasureTypeID == 2:
            for t in range(0,year): 
                NPV+= (T*v*InsuranceAmount-TotalCost)/(1+DiscountRate)**t
        else:
            RC = T*vc*(L1*TimetoRepair_with+L2*TimetoDetect_with+L3)
            for t in range(0,year): 
                NPV+= (R0-RC-TotalCost)/(1+DiscountRate)**t
    return NPV


In [95]:
CostofRepair=10
CorporateIncomeLoss=10
OrganizationProductiveLoss=3.4
CostofEquipmentReplacement=0
SLALoss=1000
IndirectLoss=0
TimetoRepair_without=16
TimetoDetect_without=0
TimesofAttack=1
TimePeriod=12
TimesofSuccessfulAttack=0.1
InitialCost=200
AnnualUpgrade=0
AnnualMaintenance=0
R_min=10
R_max=3000
L_max=5000
ReducedVulnerability=1
InsuranceAmount=3000
TimetoRepair_with=1
TimetoDetect_with=0
DiscountRate=0.01

In [96]:
L1,L2,L3,L=getLoss(CostofRepair,CorporateIncomeLoss,OrganizationProductiveLoss,CostofEquipmentReplacement,SLALoss,IndirectLoss,TimetoRepair_without,TimetoDetect_without)
R0=getRisk(TimesofAttack,TimePeriod,TimesofSuccessfulAttack,L)
TotalCost=getCost(InitialCost,AnnualUpgrade,AnnualMaintenance,TimePeriod)
MeasureTypeID=getMeasureTypeID(R0,R_min,R_max,L_max)
ROSI=calculateROSI(MeasureTypeID,TimesofAttack,TimePeriod,TimesofSuccessfulAttack,ReducedVulnerability,InsuranceAmount,TotalCost,R0,L1,L2,L3,TimetoRepair_with,TimetoDetect_with)
NPV=calculateNPV(MeasureTypeID,TimesofAttack,TimePeriod,TimesofSuccessfulAttack,ReducedVulnerability,InsuranceAmount,TotalCost,R0,L1,L2,L3,TimetoRepair_with,TimetoDetect_with,DiscountRate)

In [97]:
print('ROSI: {:.2%}'.format(ROSI))
print('NPV: {:.2f}'.format(NPV))

ROSI: -94.27%
NPV: -188.55
